In [1]:
import pandas as pd

In [11]:
cols = ["age", "workclass","fnlwgt","education","education-num","marital-status","occupation", "relationship","race","sex","capital-gain","capital-loss", "hours-per-week", "native-country","income"]

df = pd.read_csv("/home/hazim/Desktop/US-adult-census-data/input/adult.data",names=cols)

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32561 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32561 non-null  int64 
 1   workclass       32561 non-null  object
 2   fnlwgt          32561 non-null  int64 
 3   education       32561 non-null  object
 4   education-num   32561 non-null  int64 
 5   marital-status  32561 non-null  object
 6   occupation      32561 non-null  object
 7   relationship    32561 non-null  object
 8   race            32561 non-null  object
 9   sex             32561 non-null  object
 10  capital-gain    32561 non-null  int64 
 11  capital-loss    32561 non-null  int64 
 12  hours-per-week  32561 non-null  int64 
 13  native-country  32561 non-null  object
 14  income          32561 non-null  object
dtypes: int64(6), object(9)
memory usage: 3.7+ MB


In [15]:
num_cols = [
        "age",
        "fnlwgt",
        "capital-gain",
        "capital-loss",
        "hours-per-week"
    ]

In [16]:
features = [feature for feature in df.columns if feature not in ("income","kfold") and feature not in num_cols]


In [17]:
features

['workclass',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'native-country']

In [155]:
df = pd.read_csv("/home/hazim/Desktop/US-adult-census-data/input/train_folds.csv")

In [156]:
import pandas as pd
from sklearn import metrics
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import ensemble

In [157]:
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)

In [158]:
df.income.unique()

array(['<=50K', '>50K'], dtype=object)

In [159]:
num_cols = [
    "age",
    "fnlwgt",
    "capital-gain",
    "capital-loss",
    "hours-per-week"
]

df = df.drop(num_cols, axis=1)

df.loc[:, 'income'][df['income']== '<=50K'] = 0
df.loc[:, 'income'][df['income']== '>50K'] = 1

df['income'] = df.income.astype(int)


#for cols in df.columns:
   # df.loc[:, cols][df[cols]== '?'] = "abc"




In [162]:
features = [feature for feature in df.columns if feature not in ("income","kfold")]

for col in features:
    df[col] = df[col].astype(str).fillna("NONE")



In [164]:
df

,workclass,education,education-num,marital-status,occupation,relationship,race,sex,native-country,income,kfold
0,Private,HS-grad,9,Never-married,Machine-op-inspct,Own-child,Black,Male,United-States,0,0
1,Private,Some-college,10,Married-civ-spouse,Exec-managerial,Husband,White,Male,United-States,0,4
2,Local-gov,Bachelors,13,Divorced,Prof-specialty,Unmarried,White,Female,United-States,0,0
3,Private,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,United-States,0,0
4,Local-gov,9th,5,Separated,Other-service,Unmarried,Black,Female,United-States,0,1
...,...,...,...,...,...,...,...,...,...,...,...
32556,Private,Assoc-acdm,12,Never-married,Other-service,Not-in-family,White,Female,United-States,1,2
32557,?,HS-grad,9,Divorced,?,Not-in-family,Asian-Pac-Islander,Female,China,0,1
32558,Private,Some-college,10,Never-married,Exec-managerial,Not-in-family,White,Male,United-States,0,0
32559,Private,1st-4th,2,Never-married,Other-service,Own-child,White,Male,United-States,0,3


In [163]:
df_train = df[df.kfold != 0].reset_index(drop=True)

df_valid = df[df.kfold == 0].reset_index(drop=True)

ohe = preprocessing.OneHotEncoder()
#scaler = preprocessing.StandardScaler()

# fit ohe on training + validation features

full_data = pd.concat([df_train[features], df_valid[features]],axis=0)

ohe.fit(full_data[features])
x_train = ohe.transform(df_train[features])
x_valid = ohe.transform(df_valid[features])

#full_data = pd.concat([df_train[num_cols], df_valid[num_cols]],axis=0)

#scaler.fit(full_data)

#x_train_x = scaler.transform(df_train[num_cols])
#x_valid_x = scaler.transform(df_valid[num_cols])

#x_train = pd.concat([x_train,x_train_x],axis=1)
#x_valid = pd.concat([x_valid,x_valid_x],axis=1)



# modelling

clf = linear_model.LogisticRegression()

clf.fit(x_train,df_train.income.values)

valid_preds = clf.predict_proba(x_valid)[:,1]

In [165]:
auc = metrics.roc_auc_score(df_valid.income.values,valid_preds)
print(f"FOLD={0}, Accuracy = {auc}")

FOLD=0, Accuracy = 0.8747856812101785


In [166]:
x_train

<26048x118 sparse matrix of type '<class 'numpy.float64'>'
	with 234432 stored elements in Compressed Sparse Row format>

## TF Keras

In [6]:
import os
import gc
import joblib
import pandas as pd
from sklearn import metrics
from sklearn import preprocessing
import numpy as np

from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras import utils

from sklearn import linear_model
from sklearn import ensemble

In [12]:
df

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month,target,kfold
0,152722,0.0,0.0,0.0,F,N,Blue,Triangle,Lion,India,...,3.0,Contributor,Hot,i,Q,DR,3.0,11.0,0,0
1,527221,1.0,0.0,0.0,T,NaN,Blue,Circle,Axolotl,Costa Rica,...,2.0,Contributor,Boiling Hot,a,X,ne,4.0,7.0,0,0
2,87946,0.0,0.0,0.0,NaN,N,Blue,Circle,Lion,Finland,...,2.0,Novice,Warm,n,NaN,NaN,7.0,7.0,0,0
3,516876,0.0,0.0,0.0,F,N,Green,Trapezoid,Hamster,Finland,...,2.0,Expert,Hot,k,A,nS,2.0,1.0,0,0
4,522286,0.0,0.0,1.0,T,Y,Blue,Polygon,Axolotl,Canada,...,1.0,Master,Boiling Hot,NaN,M,gc,7.0,11.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
599995,515749,0.0,0.0,0.0,T,N,Blue,Polygon,Axolotl,Russia,...,1.0,Expert,Boiling Hot,f,B,DI,5.0,8.0,0,4
599996,163736,0.0,0.0,1.0,F,Y,Red,Trapezoid,Lion,NaN,...,2.0,Contributor,Cold,o,W,wa,NaN,6.0,0,4
599997,337617,0.0,0.0,0.0,T,N,Red,Trapezoid,Axolotl,Finland,...,3.0,Grandmaster,Freezing,n,M,vx,6.0,6.0,0,4
599998,72851,0.0,0.0,0.0,NaN,Y,Red,Triangle,Axolotl,Costa Rica,...,1.0,Novice,Hot,o,A,TP,5.0,11.0,0,4


In [16]:
def create_model(data,catcols):

    inputs = []
    outputs = []

    for c in catcols:
        num_unique_values = int(data[c].nunique())

        embed_dim = int(min(np.ceil((num_unique_values)/2), 50))

        inp = layers.Input(shape=(1,))

        out = layers.Embedding(
            num_unique_values + 1, embed_dim, name = c
        )(inp)


        out = layers.SpatialDropout1D(0.3)(out)

        out = layers.Reshape(target_shape = (embed_dim,))(out)

        inputs.append(inp)

        outputs.append(out)

    x = layers.Concatenate()(outputs)
    x = layers.BatchNormalization()(x)

    x = layers.Dense(300,activation = "relu")(x)
    x = layers.Dropout(0.3)(x)
    x = layers.BatchNormalization()(x)

    y = layers.Dense(2,activation="softmax")(x)

    model = Model(inputs = inputs, outputs = y)

    model.compile(loss= 'binary_crossentropy', optimizer = 'adam')
    return model


In [17]:
def run(fold):
    df = pd.read_csv("/home/hazim/Desktop/Categorical-Feature-Encoding-Challenge-II/input/train_folds.csv")

    features = [f for f in df.columns if f not in ("id", "target", "kfold")]

    for col in features:
        df[col] = df[col].astype(str).fillna("NONE")

    for feat in features:
        lbl_enc =  preprocessing.LabelEncoder()
        df[feat] = lbl_enc.fit_transform(df[feat].values)

    df_train = df[df.kfold != 0].reset_index(drop=True)
    df_valid = df[df.kfold == 0].reset_index(drop=True)

    model = create_model(df, features)

    xtrain = [
        df_train[features].values[:,k] for k in range(len(features))
    ]

    xvalid = [
        df_valid[features].values[:,k] for k in range(len(features))
    ]

    ytrain = df_train.target.values
    yvalid = df_valid.target.values

    ytrain_cat = utils.to_categorical(ytrain)
    yvalid_cat = utils.to_categorical(yvalid)

    model.fit(xtrain, ytrain_cat, validation_data = (xvalid,yvalid_cat), verbose =1, batch_size = 1024, epochs = 3)

    valid_preds = model.predict(xvalid)[:,1]

    print(metrics.roc_auc_score(yvalid,valid_preds))

    K.clear_session()





In [18]:
for i in range(4):
    run(i)

Epoch 1/3
469/469 [==============================] - 33s 70ms/step - loss: 0.4725 - val_loss: 0.4309
Epoch 2/3
469/469 [==============================] - 31s 66ms/step - loss: 0.4077 - val_loss: 0.4001
Epoch 3/3
469/469 [==============================] - 32s 68ms/step - loss: 0.4009 - val_loss: 0.3995
0.7834164338137872
Epoch 1/3
469/469 [==============================] - 33s 70ms/step - loss: 0.4727 - val_loss: 0.4288
Epoch 2/3
469/469 [==============================] - 32s 68ms/step - loss: 0.4079 - val_loss: 0.3999
Epoch 3/3
469/469 [==============================] - 32s 68ms/step - loss: 0.4008 - val_loss: 0.3995
0.7829379332683242
Epoch 1/3
469/469 [==============================] - 33s 71ms/step - loss: 0.4732 - val_loss: 0.4277
Epoch 2/3
469/469 [==============================] - 9s 19ms/step - loss: 0.4075 - val_loss: 0.4006
Epoch 3/3
469/469 [==============================] - 26s 55ms/step - loss: 0.4009 - val_loss: 0.3996
0.7835192553528925
Epoch 1/3
469/469 [================